<a href="https://colab.research.google.com/github/AronAhn/tf_udemy/blob/master/TF_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer: 
https://medium.com/ml-book/demonstration-of-tensorflow-feature-columns-tf-feature-column-3bfcca4ca5c4

In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import pandas as pd

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### regression

In [0]:
housing =  pd.read_csv("/content/drive/My Drive/udemy_tf/FULL-TENSORFLOW-NOTES-AND-DATA/02-TensorFlow-Basics/cal_housing_clean.csv")

In [0]:
y_val = housing['medianHouseValue']
x_data = housing.drop('medianHouseValue', axis=1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_val, test_size=0.3, random_state=101)

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(data=scaler.transform(X_train), 
                       columns=X_train.columns,
                       index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test),
                      columns=X_test.columns,
                      index=X_test.index)

In [0]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

feat_cols = [age, rooms, bedrooms, pop, households, income]

In [0]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True
)

In [62]:
model = tf.compat.v1.estimator.DNNRegressor(
    hidden_units = [4,4,4], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmplpfeqxyw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b797d5c88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [73]:
model.train(input_fn=input_func, steps=20000)

INFO:tensorflow:global_step/sec: 271.566
INFO:tensorflow:loss = 60043700000.0, step = 20200 (0.363 sec)
INFO:tensorflow:global_step/sec: 274.941
INFO:tensorflow:loss = 38000845000.0, step = 20300 (0.363 sec)
INFO:tensorflow:global_step/sec: 274.245
INFO:tensorflow:loss = 25305526000.0, step = 20400 (0.372 sec)
INFO:tensorflow:global_step/sec: 239.039
INFO:tensorflow:loss = 109463810000.0, step = 20500 (0.419 sec)
INFO:tensorflow:global_step/sec: 257.996
INFO:tensorflow:loss = 73656150000.0, step = 20600 (0.385 sec)
INFO:tensorflow:global_step/sec: 288.814
INFO:tensorflow:loss = 116013090000.0, step = 20700 (0.337 sec)
INFO:tensorflow:global_step/sec: 292.609
INFO:tensorflow:loss = 91915880000.0, step = 20800 (0.353 sec)
INFO:tensorflow:global_step/sec: 276.662
INFO:tensorflow:loss = 52695100000.0, step = 20900 (0.357 sec)
INFO:tensorflow:global_step/sec: 277.653
INFO:tensorflow:loss = 114409650000.0, step = 21000 (0.355 sec)
INFO:tensorflow:global_step/sec: 285.466
INFO:tensorflow:loss

In [64]:
predict_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_test, batch_size=10, num_epochs=1, shuffle=False
)
pred_gen = model.predict(predict_input_func)
prediction = list(pred_gen)

final_preds = [pred['predictions'] for pred in prediction]

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmplpfeqxyw/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [65]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, final_preds)**.5

527430337.8177329

In [56]:
mean_squared_error(y_test, final_preds)**.5

516019371.5056406

### classification

In [0]:
census = pd.read_csv('/content/drive/My Drive/udemy_tf/FULL-TENSORFLOW-NOTES-AND-DATA/02-TensorFlow-Basics/census_data.csv')

In [0]:
def label_fix(label):
  if label==' <=50K':
    return 0
  else: 
    return 1
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [0]:
x_data = census.drop('income_bracket', axis=1)
y_label = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.3, random_state=101)

In [71]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [0]:
gender = tf.compat.v1.feature_column.categorical_column_with_vocabulary_list("gender", ['Female', 'Male'])
occupation = tf.compat.v1.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.compat.v1.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.compat.v1.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.compat.v1.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.compat.v1.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.compat.v1.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [0]:
age = tf.compat.v1.feature_column.numeric_column("age")
education_num = tf.compat.v1.feature_column.numeric_column("education_num")
capital_gain = tf.compat.v1.feature_column.numeric_column("capital_gain")
capital_loss = tf.compat.v1.feature_column.numeric_column("capital_loss")
hours_per_week = tf.compat.v1.feature_column.numeric_column("hours_per_week")

In [0]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country,
             age, education_num, capital_gain, capital_loss, hours_per_week]

In [88]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_train, y=y_train, batch_size=100, num_epochs=None, shuffle=True)
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)
model.train(input_fn=input_func, steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmppz311p2p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b79a4f8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` metho

In [105]:
pred_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_test, batch_size=len(X_test), shuffle=False)
pred_gen = model.predict(input_fn=pred_fn)
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppz311p2p/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [107]:
from sklearn.metrics import classification_report
final_preds = [pred['class_ids'][0] for pred in predictions]
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      7436
           1       0.70      0.62      0.66      2333

    accuracy                           0.85      9769
   macro avg       0.79      0.77      0.78      9769
weighted avg       0.84      0.85      0.84      9769

